In [19]:
# import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
# import lxml
# import xmltodict
import collections
from collections import OrderedDict
import ast
from pandarallel import pandarallel
pandarallel.initialize()

headers={
    'accept': '*/*',
#     'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [20]:
sitemaps = [d.strftime('%Y/%m/%d') for d in pd.date_range('1999/01/01','2021/04/21')]
df = pd.DataFrame(sitemaps, columns = ['date'])
df['page'] = 'https://nypost.com/' +df.date
df

,date,page
0,1999/01/01,https://nypost.com/1999/01/01
1,1999/01/02,https://nypost.com/1999/01/02
2,1999/01/03,https://nypost.com/1999/01/03
3,1999/01/04,https://nypost.com/1999/01/04
4,1999/01/05,https://nypost.com/1999/01/05
...,...,...
8142,2021/04/17,https://nypost.com/2021/04/17
8143,2021/04/18,https://nypost.com/2021/04/18
8144,2021/04/19,https://nypost.com/2021/04/19
8145,2021/04/20,https://nypost.com/2021/04/20


In [35]:
def extract_article(article):
    href = article.a['href']
    attributes = article.attrs['class']
    tags = filter(lambda x: 'tag' in x, attributes)
    tags = [i.replace('tag-','').replace('-',' ') for i in tags]

    section = filter(lambda x: 'section' in x, attributes)
    section = [i.replace('section-','').replace('-',' ') for i in section]

    head = article.find_all('div', class_="entry-header")[0]
    href = head.a['href']
    title = re.sub("^\s+|\s+$", "", head.text, flags=re.UNICODE).split('\n')[0]
    date = re.search("\n([\w ,]+) \|",head.text)[1]

    result ={'title': title, 'date': date, 'tags': tags, 'section': section, 'url': href}
    return result

def get_articles(page):
    try:
        coverpage = requests.get(page,headers = headers).content
        soup = BeautifulSoup(coverpage)
        body = soup.find_all('div',class_= 'site-content box')[0]
        articles = body.find_all('li')
        results = []
        for article in articles:
            result = extract_article(article)
            results.append(result)       
        return results
    except:
        return

In [36]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size= 16*5
i = 0
while i < df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1,'articles']=\
    df.loc[i: i+ batch_size-1, 'page'].parallel_apply(get_articles).values
    i += batch_size

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,80,160,240,320,400,480,560,640,720,800,880,960,1040,1120,1200,1280,1360,1440,1520,1600,1680,1760,1840,1920,2000,2080,2160,2240,2320,2400,2480,2560,2640,2720,2800,2880,2960,3040,3120,3200,3280,3360,3440,3520,3600,3680,3760,3840,3920,4000,4080,4160,4240,4320,4400,4480,4560,4640,4720,4800,4880,4960,5040,5120,5200,5280,5360,5440,5520,5600,5680,5760,5840,5920,6000,6080,6160,6240,6320,6400,6480,6560,6640,6720,6800,6880,6960,7040,7120,7200,7280,7360,7440,7520,7600,7680,7760,7840,7920,8000,8080,

In [38]:
df

,date,page,articles
0,1999/01/01,https://nypost.com/1999/01/01,"[{'title': 'IN '98, SHOWSGOT UPSTAGED', 'date': 'January 1, 1999', 'tags': [], 'section': ['entertainment'], 'url': 'https://nypost.com/1999/01/01/in-98-showsgot-upstaged/'}, {'title': 'ACTOR-DIRECTOR & DRIFTER DIE', 'date': 'January 1, 1999', 'tags': [], 'section': ['entertainment'], 'url': 'https://nypost.com/1999/01/01/actor-director-drifter-die/'}, {'title': 'TOP STATE AIDE IN MORALS BUST', 'date': 'January 1, 1999', 'tags': [], 'section': ['news'], 'url': 'https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/'}, {'title': 'ICY HUDSON CRAMPS BRR-AVE SWIMMER'S STYLE', 'date': 'Ja..."
1,1999/01/02,https://nypost.com/1999/01/02,"[{'title': 'TV EN ESPANOL GETS HOT: FASTEST GROWING SEGMENT OF POP', 'date': 'January 2, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/02/tv-en-espanol-gets-hot-fastest-growing-segment-of-pop/'}, {'title': 'GAMESVILLE'S PLAYERS AD UP EVERY 10 MINS.', 'date': 'January 2, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/02/gamesvilles-players-ad-up-every-10-mins/'}, {'title': 'THE INTERNET GIVETH - AND TAKES BACK', 'date': 'January 2, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/02/the-internet-giveth-and-takes-back/'}, {'tit..."
2,1999/01/03,https://nypost.com/1999/01/03,"[{'title': 'ELVIS LIVES IN TWO-PART BIOGRAPHY', 'date': 'January 3, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/03/elvis-lives-in-two-part-biography/'}, {'title': 'BANK STOCKS SHOULD BOUNCE BACK IN '99', 'date': 'January 3, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/03/bank-stocks-should-bounce-back-in-99/'}, {'title': 'SOME CYBER-SLANG HAS A VERY SHORT SHELF LIFE', 'date': 'January 3, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/03/some-cyber-slang-has-a-very-short-shelf-life/'}, {'title': 'WIN, PLACE AND SHOWS', '..."
3,1999/01/04,https://nypost.com/1999/01/04,"[{'title': 'STEADY AS SHE GOES; GROWTH SIGNS STILL GOOD, DOUBTS ASIDE', 'date': 'January 4, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/04/steady-as-she-goes-growth-signs-still-good-doubts-aside/'}, {'title': 'WHAT'S AHEAD FOR THE YEAR IN MEDIA: STAY TUNED', 'date': 'January 4, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/04/whats-ahead-for-the-year-in-media-stay-tuned/'}, {'title': 'TV TICKER', 'date': 'January 4, 1999', 'tags': [], 'section': ['entertainment', 'tv'], 'url': 'https://nypost.com/1999/01/04/tv-ticker/'}, {'title': 'BETTER 'LA..."
4,1999/01/05,https://nypost.com/1999/01/05,"[{'title': 'STORM BLOWS BY NIAGARA', 'date': 'January 5, 1999', 'tags': [], 'section': ['sports'], 'url': 'https://nypost.com/1999/01/05/storm-blows-by-niagara/'}, {'title': 'HARVEY WAKES UP RANGERS', 'date': 'January 5, 1999', 'tags': [], 'section': ['sports'], 'url': 'https://nypost.com/1999/01/05/harvey-wakes-up-rangers/'}, {'title': 'DON'T COUNT OUT INVESTMENTS IN EUROLAND', 'date': 'January 5, 1999', 'tags': [], 'section': [], 'url': 'https://nypost.com/1999/01/05/dont-count-out-investments-in-euroland/'}, {'title': 'INTUIT EXEC BLASTS MICROSOFT 'CHOKE HOLD'', 'date': 'January 5, 1999..."
...,...,...,...
8142,2021/04/17,https://nypost.com/2021/04/17,"[{'title': 'Grace and grief as Prince Philip is laid to rest', 'date': 'April 17, 2021', 'tags': ['kate middleton', 'meghan markle', 'prince andrew', 'prince charles', 'prince harry', 'prince philip', 'prince william', 'queen elizabeth ii', 'royal family'], 'section': ['news'], 'url': 'https://nypost.com/2021/04/17/grace-and-grief-as-prince-philip-is-laid-to-rest/'}, {'title': 'Yankees' experiment with this veteran appears over', 'date': 'April 17, 2021', 'tags': ['aaron boone', 'dj lemahieu', 'jay bruce', 'new york yankees', 'rougned odor'], 'section': ['baseball'], 'url': 'https://nypost..."
8143,2021/04/18,https://nypost.com/2021/04/18,"[{'title': 'Penske Media buys big stake in S

In [39]:
df.to_csv('nypost/nypost_1.csv', index=False)

In [3]:
df= pd.read_csv('nypost/nypost_1.csv', header=0)
df

,date,page,articles
0,1999/01/01,https://nypost.com/1999/01/01,"['<li class=""article full article-loop__article post-17 type-article status-publish section-entertainment"" id=""post-17"">\n<div class=""entry-header"">\n<h3 class=""entry-heading"">\n<a class=""postid-17"" href=""https://nypost.com/1999/01/01/in-98-showsgot-upstaged/"">\n\t\t\t\t\t\t\t\tIN \'98, SHOWSGOT UPSTAGED</a>\n</h3>\n<div class=""entry-meta"">\n<p>January 1, 1999 | 5:00am</p>\n</div><!-- .entry-meta -->\n</div><!-- .entry-header -->\n<div class=""entry-content"">\n\t\t\t\t\t\tAH, the theater – where what happens offstage is often more entertaining than what happens on. For theater mavens who la..."
1,1999/01/02,https://nypost.com/1999/01/02,"['<li class=""article full article-loop__article post-240 type-article status-publish"" id=""post-240"">\n<div class=""entry-header"">\n<h3 class=""entry-heading"">\n<a class=""postid-240"" href=""https://nypost.com/1999/01/02/tv-en-espanol-gets-hot-fastest-growing-segment-of-pop/"">\n\t\t\t\t\t\t\t\tTV EN ESPANOL GETS HOT: FASTEST GROWING SEGMENT OF POP</a>\n</h3>\n<div class=""entry-meta"">\n<p>January 2, 1999 | 5:00am</p>\n</div><!-- .entry-meta -->\n</div><!-- .entry-header -->\n<div class=""entry-content"">\n\t\t\t\t\t\tABSTRACT: Spanish TV in the U.S. is finally having its day in the sun. Media comp..."
2,1999/01/03,https://nypost.com/1999/01/03,"['<li class=""article full article-loop__article post-323 type-article status-publish"" id=""post-323"">\n<div class=""entry-header"">\n<h3 class=""entry-heading"">\n<a class=""postid-323"" href=""https://nypost.com/1999/01/03/elvis-lives-in-two-part-biography/"">\n\t\t\t\t\t\t\t\tELVIS LIVES IN TWO-PART BIOGRAPHY</a>\n</h3>\n<div class=""entry-meta"">\n<p>January 3, 1999 | 5:00am</p>\n</div><!-- .entry-meta -->\n</div><!-- .entry-header -->\n<div class=""entry-content"">\n\t\t\t\t\t\tELVIS lives. At least that’s what Little, Brown predicts for its second volume of a two-part series on Elvis Presley by Pe..."
3,1999/01/04,https://nypost.com/1999/01/04,"['<li class=""article full article-loop__article post-475 type-article status-publish"" id=""post-475"">\n<div class=""entry-header"">\n<h3 class=""entry-heading"">\n<a class=""postid-475"" href=""https://nypost.com/1999/01/04/steady-as-she-goes-growth-signs-still-good-doubts-aside/"">\n\t\t\t\t\t\t\t\tSTEADY AS SHE GOES; GROWTH SIGNS STILL GOOD, DOUBTS ASIDE</a>\n</h3>\n<div class=""entry-meta"">\n<p>January 4, 1999 | 5:00am</p>\n</div><!-- .entry-meta -->\n</div><!-- .entry-header -->\n<div class=""entry-content"">\n\t\t\t\t\t\tMost top Wall Street strategists expect corporate profits to decline next ye..."
4,1999/01/05,https://nypost.com/1999/01/05,"['<li class=""article full article-loop__article post-729 type-article status-publish section-sports"" id=""post-729"">\n<div class=""entry-header"">\n<h3 class=""entry-heading"">\n<a class=""postid-729"" href=""https://nypost.com/1999/01/05/storm-blows-by-niagara/"">\n\t\t\t\t\t\t\t\tSTORM BLOWS BY NIAGARA</a>\n</h3>\n<div class=""entry-meta"">\n<p>January 5, 1999 | 5:00am</p>\n</div><!-- .entry-meta -->\n</div><!-- .entry-header -->\n<div class=""entry-content"">\n\t\t\t\t\t\tSJU exacts revenge for loss last season They all said it would be different. Every St. John’s player that had to sit through last..."
...,...,...,...
8142,2021/04/17,https://nypost.com/2021/04/17,"['<li class=""article half article-loop__article post-17998522 type-article status-publish has-post-thumbnail tag-kate-middleton tag-meghan-markle tag-prince-andrew tag-prince-charles tag-prince-harry tag-prince-philip tag-prince-william tag-queen-elizabeth-ii tag-royal-family section-news"" id=""post-17998522"">\n<div class=""entry-thumbnail"">\n<a href=""https://nypost.com/2021/04/17/grace-and-grief-as-prince-philip-is-laid-to-rest/"" tabindex=""-1"">\n<picture class=""entry-thumbnail"">\n<source data-sizes="""" data-srcset=""https://nypost.com/wp-content/uploads/sites/2/2021/04/prince-charles-philip-f.

In [41]:
df = df[~df.articles.isnull()].reset_index(drop=True)
# df[df.articles.str.len()==4]

In [42]:
df = df.explode('articles').reset_index(drop=True)
# df[~df.articles.str.startswith('<li')]
df

,date,page,articles
0,1999/01/01,https://nypost.com/1999/01/01,"{'title': 'IN '98, SHOWSGOT UPSTAGED', 'date': 'January 1, 1999', 'tags': [], 'section': ['entertainment'], 'url': 'https://nypost.com/1999/01/01/in-98-showsgot-upstaged/'}"
1,1999/01/01,https://nypost.com/1999/01/01,"{'title': 'ACTOR-DIRECTOR & DRIFTER DIE', 'date': 'January 1, 1999', 'tags': [], 'section': ['entertainment'], 'url': 'https://nypost.com/1999/01/01/actor-director-drifter-die/'}"
2,1999/01/01,https://nypost.com/1999/01/01,"{'title': 'TOP STATE AIDE IN MORALS BUST', 'date': 'January 1, 1999', 'tags': [], 'section': ['news'], 'url': 'https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/'}"
3,1999/01/01,https://nypost.com/1999/01/01,"{'title': 'ICY HUDSON CRAMPS BRR-AVE SWIMMER'S STYLE', 'date': 'January 1, 1999', 'tags': [], 'section': ['news'], 'url': 'https://nypost.com/1999/01/01/icy-hudson-cramps-brr-ave-swimmers-style/'}"
4,1999/01/01,https://nypost.com/1999/01/01,"{'title': 'IRAQ MOCKS 'EVIL ADULTERER' CLINTON', 'date': 'January 1, 1999', 'tags': [], 'section': ['news'], 'url': 'https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/'}"
...,...,...,...
1034120,2021/04/21,https://nypost.com/2021/04/21,"{'title': 'Xi to attend Biden's climate change summit in first meeting of two leaders', 'date': 'April 21, 2021', 'tags': ['china', 'climate change', 'joe biden', 'united states', 'xi jinping'], 'section': ['news'], 'url': 'https://nypost.com/2021/04/21/xi-to-attend-bidens-climate-change-summit-in-first-meeting-of-two-leaders/'}"
1034121,2021/04/21,https://nypost.com/2021/04/21,"{'title': 'TikTok faces claim for billions in London child privacy lawsuit', 'date': 'April 21, 2021', 'tags': ['data breach', 'european union', 'london', 'tiktok', 'united kingdom'], 'section': ['tech'], 'url': 'https://nypost.com/2021/04/21/tiktok-faces-claim-for-billions-in-london-child-privacy-lawsuit/'}"
1034122,2021/04/21,https://nypost.com/2021/04/21,"{'title': 'Yankees call-up does it all in place of abrupt retiree ', 'date': 'April 21, 2021', 'tags': ['jay bruce', 'mike ford', 'new york yankees'], 'section': ['baseball'], 'url': 'https://nypost.com/2021/04/21/mike-ford-does-it-all-for-yankees-in-place-of-jay-bruce/'}"
1034123,2021/04/21,https://nypost.com/2021/04/21,"{'title': 'Cops reveal shocking footage in fatal shooting of knife-wielding teen girl', 'date': 'April 21, 2021', 'tags': ['makhia bryant', 'ohio', 'police body cameras', 'police shootings'], 'section': ['news'], 'url': 'https://nypost.com/2021/04/21/bodycam-footage-released-of-columbus-officers-fatal-shooting-of-teen-girl/'}"


In [48]:
df1 = pd.json_normalize(df.articles)
df1

,title,date,tags,section,url
0,"IN '98, SHOWSGOT UPSTAGED","January 1, 1999",[],[entertainment],https://nypost.com/1999/01/01/in-98-showsgot-upstaged/
1,ACTOR-DIRECTOR & DRIFTER DIE,"January 1, 1999",[],[entertainment],https://nypost.com/1999/01/01/actor-director-drifter-die/
2,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/
3,ICY HUDSON CRAMPS BRR-AVE SWIMMER'S STYLE,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/icy-hudson-cramps-brr-ave-swimmers-style/
4,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/
...,...,...,...,...,...
1034120,Xi to attend Biden's climate change summit in first meeting of two leaders,"April 21, 2021","[china, climate change, joe biden, united states, xi jinping]",[news],https://nypost.com/2021/04/21/xi-to-attend-bidens-climate-change-summit-in-first-meeting-of-two-leaders/
1034121,TikTok faces claim for billions in London child privacy lawsuit,"April 21, 2021","[data breach, european union, london, tiktok, united kingdom]",[tech],https://nypost.com/2021/04/21/tiktok-faces-claim-for-billions-in-london-child-privacy-lawsuit/
1034122,Yankees call-up does it all in place of abrupt retiree,"April 21, 2021","[jay bruce, mike ford, new york yankees]",[baseball],https://nypost.com/2021/04/21/mike-ford-does-it-all-for-yankees-in-place-of-jay-bruce/
1034123,Cops reveal shocking footage in fatal shooting of knife-wielding teen girl,"April 21, 2021","[makhia bryant, ohio, police body cameras, police shootings]",[news],https://nypost.com/2021/04/21/bodycam-footage-released-of-columbus-officers-fatal-shooting-of-teen-girl/


In [49]:
df1.to_csv('nypost_all_links.csv', index=False)

# 可以用标签过滤

In [185]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [80]:
from collections import Counter

head = df1['title'] + ' '+ df1['url'] + ' ' +df1['tags'].apply(str) + ' ' + df1['section'].apply(str)

df1['head_in'] = head.parallel_apply(find_relevant)
df1['head_out'] = head.parallel_apply(find_exclude)

head_in = df1['head_in'].str.len()
head_out = df1['head_out'].str.len()

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df1[remove_index== True].index
remove_index.shape

(5683,)

In [83]:
df1 = df1[df1.head_out.str.len()==0].reset_index(drop=True)
# df1[df1.head_out.str.len()==1]
df1

,title,date,tags,section,url,head_in,head_out
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,{},{}
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,{},{}
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'terror': 2, 'threat': 2}",{}
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],[news],https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,{},{}
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],[news],https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/,{},{}
...,...,...,...,...,...,...,...
446260,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","[makhia bryant, ohio, police body cameras, police brutality, police shootings, teens]",[news],https://nypost.com/2021/04/21/mom-of-ohio-teen-girl-fatally-shot-by-cop-speaks-out/,"{'shot': 2, 'police': 3, 'shooting': 1}",{}
446261,Derek Chauvin taken to maximum-security prison after murder conviction,"April 21, 2021","[derek chauvin, george floyd, minneapolis, prisons]",[news],https://nypost.com/2021/04/21/derek-chauvin-taken-to-minnesota-correctional-facility-oak-park-heights-maximum-security-prison/,"{'security': 2, 'prison': 3, 'murder': 1, 'convict': 1}",{}
446262,Columbus protesters gather at police headquarters over shooting of teen,"April 21, 2021","[makhia bryant, ohio, police shootings, protests]",[news],https://nypost.com/2021/04/21/protesters-gather-at-columbus-police-station-over-death-of-teen/,"{'protest': 3, 'police': 3, 'shooting': 2, 'death': 1}",{}
446263,J&J to resume COVID-19 vaccine rollout in Europe,"April 21, 2021","[covid vaccine, europe, johnson johnson]",[news],https://nypost.com/2021/04/21/jj-will-resume-covid-19-vaccine-rollout-in-europe/,{},{}


In [88]:
df1.section.apply(str).value_counts()

['news']                          244038
['metro']                         111962
['opinion']                        37005
[]                                 22618
['living']                         20794
['media']                           3131
['metro', 'news']                   1935
['college']                         1525
['living', 'news']                  1247
['media', 'news']                    661
['photos']                           582
['living', 'metro']                  350
['media', 'metro']                    86
['living', 'opinion']                 84
['living', 'media']                   61
['news', 'opinion']                   51
['living', 'metro', 'news']           29
['metro', 'opinion']                  25
['news', 'photos']                    17
['media', 'metro', 'news']            14
['media', 'opinion']                   9
['college', 'news']                    5
['living', 'media', 'news']            5
['living', 'metro', 'opinion']         4
['living', 'phot

In [86]:
df1.tags.apply(str).value_counts()[0:60]

[]                                                                            264882
['blotter']                                                                     2801
['weird but true']                                                              2770
['daily blotter']                                                                959
['sex and relationships']                                                        809
['jobs']                                                                         746
['food']                                                                         559
['day in photos']                                                                137
['books', 'in my library']                                                       115
['brooklyn', 'shootings']                                                         97
['crime']                                                                         82
['anders behring breivik']                                       

In [139]:
df1 = df1[~df1.section.apply(str).str.contains('living|college')]
df1

,title,date,tags,section,url,story
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,"{'tag2': None, 'section2': None, 'text': 'ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and nea..."
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,"{'tag2': None, 'section2': None, 'text': 'Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Ma..."
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'tag2': None, 'section2': None, 'text': 'JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanza..."
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,"{'tag2': None, 'section2': None, 'text': ""New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your th..."
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/,"{'tag2': None, 'section2': None, 'text': 'Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the c..."
...,...,...,...,...,...,...
423677,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","['makhia bryan

In [92]:
df1.tags.apply(str).value_counts()[0:60]

[]                                                                                  264625
['blotter']                                                                           2801
['weird but true']                                                                    2770
['daily blotter']                                                                      959
['day in photos']                                                                      137
['brooklyn', 'shootings']                                                               97
['crime']                                                                               82
['anders behring breivik']                                                              75
['andrew cuomo', 'coronavirus', 'coronavirus in ny']                                    65
['nypd']                                                                                64
['brooklyn', 'crime']                                                                   64

In [93]:
df1.to_csv('nypost_2.csv', index=False)

In [98]:
df1 = df1.iloc[:, :5]
df1

,title,date,tags,section,url
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],[news],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],[news],https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],[news],https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/
...,...,...,...,...,...
446260,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","[makhia bryant, ohio, police body cameras, police brutality, police shootings, teens]",[news],https://nypost.com/2021/04/21/mom-of-ohio-teen-girl-fatally-shot-by-cop-speaks-out/
446261,Derek Chauvin taken to maximum-security prison after murder conviction,"April 21, 2021","[derek chauvin, george floyd, minneapolis, prisons]",[news],https://nypost.com/2021/04/21/derek-chauvin-taken-to-minnesota-correctional-facility-oak-park-heights-maximum-security-prison/
446262,Columbus protesters gather at police headquarters over shooting of teen,"April 21, 2021","[makhia bryant, ohio, police shootings, protests]",[news],https://nypost.com/2021/04/21/protesters-gather-at-columbus-police-station-over-death-of-teen/
446263,J&J to resume COVID-19 vaccine rollout in Europe,"April 21, 2021","[covid vaccine, europe, johnson johnson]",[news],https://nypost.com/2021/04/21/jj-will-resume-covid-19-vaccine-rollout-in-europe/


In [109]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    result ={'tag2': None, 'section2': None, 'text': None}
    try:
        article = Article(url,config= config)
        article.download()
        time.sleep(1)
        article.parse()
        
        text = re.sub("^\s+|\s+$", "", article.text, flags=re.UNICODE)
          
        try:
            find_sec = soup.find_all('p', class_= 'section-tag')[0]
            section = find_sec.find_all('a', href=True)
            section = section[0].text
            section = re.sub("^\s+|\s+$", "", section, flags=re.UNICODE) 
        except:
            section = None    
        try:
            find_tag = soup.find_all('div', class_= 'tag-list')[0]
            tags = find_tag.find_all('a', href=True)
            tags = [re.sub("^\s+|\s+$", "", i.text, flags=re.UNICODE) for i in tags]
        except:
            tags = None
 
        result ={'tag2': tags, 'section2': section, 'text': text} 
    except:
        print('error %s'%url)
    return result

# 正在爬

In [114]:
df1 = df1.reset_index(drop=True)

In [116]:
pandarallel.initialize(nb_workers=32, progress_bar=False)
batch_size= 32*10
i = resume = 0
while i < df1.shape[0]:
    print(i, end=',')
    df1.loc[i: i+ batch_size-1,'story']=\
    df1.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_story).values
    i += batch_size
    if (i-resume) % (batch_size * 50):
        df1.to_csv('nypost/nypost_3.csv', index=False)


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,320,640,960,1280,error https://nypost.com/1999/02/24/big-boards-new-math-12-%c2%bd-hours-will-equal-30-mins/
1600,1920,2240,2560,2880,3200,3520,3840,4160,4480,4800,5120,5440,5760,6080,6400,6720,7040,7360,7680,8000,8320,8640,8960,9280,9600,9920,10240,10560,10880,11200,11520,11840,12160,12480,12800,13120,13440,13760,14080,14400,14720,15040,15360,15680,16000,16320,16640,16960,17280,17600,17920,18240,18560,18880,19200,19520,19840,20160,20480,20800,21120,21440,21760,22080,22400,22720,23040,23360,23680,24000,24320,24640,24960,25280,25600,25920,26240,26560,26880,27200,27520,27840,28160,28480,28800,29120,29440,29760,30080,30400,30720,31040,31360,31680,32000,32320,32640,32960,33280,33600,33920,34240,34560,34880,35200,35520,35840,36160,36480,36800,37120,37440,37760,38080,38400,38720,error https://nypost.com/2002/03/08/freedom-rings-for-louima-cop-schwar

NameError: name 'idx' is not defined

In [117]:
df1.to_csv('nypost/nypost_3.csv', index=False)

In [138]:
df1=pd.read_csv('nypost/nypost_3.csv',header=0)

In [140]:
df1.section.apply(str).unique()

array(["['news']", '[]', "['opinion']", "['media']", "['metro']",
       "['metro', 'news']", "['photos']", "['metro', 'opinion']",
       "['media', 'news']", "['media', 'opinion']", "['news', 'photos']",
       "['news', 'opinion']", "['metro', 'news', 'opinion']",
       "['media', 'metro']", "['media', 'metro', 'news']",
       "['metro', 'photos']", "['media', 'photos']", "['news', 'video']",
       "['media', 'news', 'opinion']", "['promotions']", "['video']",
       "['opinion', 'preview']", "['news', 'preview']",
       "['opinion', 'photos']"], dtype=object)

In [141]:
df1.tags.apply(str).value_counts()[0:60]

[]                                                                                  264257
['blotter']                                                                           2801
['weird but true']                                                                    2770
['daily blotter']                                                                      959
['day in photos']                                                                      137
['brooklyn', 'shootings']                                                               97
['crime']                                                                               82
['anders behring breivik']                                                              75
['andrew cuomo', 'coronavirus', 'coronavirus in ny']                                    65
['brooklyn', 'crime']                                                                   64
['nypd']                                                                                64

In [142]:
df1

,title,date,tags,section,url,story
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,"{'tag2': None, 'section2': None, 'text': 'ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and nea..."
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,"{'tag2': None, 'section2': None, 'text': 'Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Ma..."
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'tag2': None, 'section2': None, 'text': 'JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanza..."
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,"{'tag2': None, 'section2': None, 'text': ""New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your th..."
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/,"{'tag2': None, 'section2': None, 'text': 'Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the c..."
...,...,...,...,...,...,...
423677,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","['makhia bryan

In [151]:
df1= df1.reset_index(drop=True)

In [145]:
story = pd.json_normalize(df1.story.apply(eval))
story     

,tag2,section2,text
0,None,None,"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res..."
1,None,None,"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a..."
2,None,None,"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w..."
3,None,None,"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai..."
4,None,None,"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta..."
...,...,...,...
422146,None,None,"The mother of the 16-year-old Columbus, Ohio, girl who was shot by police said in a tearful interview that her “loving, peaceful” daughter called the cops “for protection — not to be a homicide.”\n\nMa’Khia Bryant was fatally shot after her family said she called police to report girls fighting about 4:30 p.m. Tuesday, shortly before former Minneapolis cop Derek Chauvin was found guilty of murdering George Floyd.\n\nVideo released by police shows an officer encountering the teen, who appears to be armed with a knife and chasing another girl, who falls in front of the cop.\n\nThe officer sc..."
422147,None,None,"Derek Chauvin is pictured in a new booking photo released Wednesday. MDC\n\nAfter being convicted in the murder of George Floyd, Derek Chauvin was taken to the maximum-security Minnesota Correctional Facility-Oak Park Heights, where he was put on “administr

In [153]:
df1 = pd.concat([df1, story], axis=1)
df1

,title,date,tags,section,url,story,tag2,section2,text
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,"{'tag2': None, 'section2': None, 'text': 'ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and nea...",None,None,"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res..."
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,"{'tag2': None, 'section2': None, 'text': 'Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Ma...",None,None,"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a..."
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'tag2': None, 'section2': None, 'text': 'JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanza...",None,None,"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings o

In [154]:
df1= df1.drop(columns='story')
df1

,title,date,tags,section,url,tag2,section2,text
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,None,None,"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res..."
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,None,None,"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a..."
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,None,None,"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w..."
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,None,None,"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai..."
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/,None,None,"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta..."
...,...,...,...,...,...,...,...,...
422146,'I want answers': Mom of tee

In [164]:
df1.columns

Index(['title', 'date', 'tags', 'section', 'url', 'tag2', 'section2', 'text'], dtype='object')

In [165]:
df1 = df1[['title', 'date', 'tags', 'section', 'text', 'url']]
df1

,title,date,tags,section,text,url
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res...",https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a...",https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w...",https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai...",https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta...",https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/
...,...,...,...,...,...,...
422146,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","['makhia bryant'

In [169]:
df1 = df1[df1.text.str.len()>100]
df1

,title,date,tags,section,text,url
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res...",https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a...",https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w...",https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai...",https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta...",https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/
...,...,...,...,...,...,...
422146,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","['makhia bryant'

In [171]:
df1 = df1[~df1.text.duplicated()]
df1

,title,date,tags,section,text,url
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res...",https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/
1,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a...",https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/
2,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w...",https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/
3,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai...",https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta...",https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/
...,...,...,...,...,...,...
422146,'I want answers': Mom of teen girl fatally shot by cop speaks out,"April 21, 2021","['makhia bryant'

In [172]:
df1.to_csv('nypost_1999_2021.csv', index=False)

In [226]:
df1= pd.read_csv('nypost_1999_2021.csv', header=0)

In [227]:
df1.tags.apply(str).value_counts()[0:60]

[]                                                                                  258093
['blotter']                                                                           2799
['weird but true']                                                                    2769
['daily blotter']                                                                      958
['day in photos']                                                                      136
['brooklyn', 'shootings']                                                               97
['crime']                                                                               81
['anders behring breivik']                                                              75
['andrew cuomo', 'coronavirus', 'coronavirus in ny']                                    65
['nypd']                                                                                63
['brooklyn', 'crime']                                                                   63

In [228]:
df1= df1[~df1.tags.apply(str).str.contains('weird but true|living|college')]
df1 = df1[~df1.url.str.contains('https://bit.ly')]

In [229]:
from collections import Counter

head = df1['title'] + ' '+ df1['url'] + ' ' +df1['tags'].apply(str) + ' ' + df1['section'].apply(str) + df1['text'].apply(lambda x: x[0:100])

df1['head_in'] = head.parallel_apply(find_relevant)
df1['head_out'] = head.parallel_apply(find_exclude)

# df1['text_in'] = df1[' text'].parallel_apply(find_relevant)
# df1['text_out'] = df1['text'].parallel_apply(find_exclude)

head_in = df1['head_in'].str.len()
head_out = df1['head_out'].str.len()

# body_in = df1['text_in'].str.len()
# body_out = df1['text_out'].str.len()

In [221]:
remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1)) #|\
#     (body_in < 2) |\
#     ((body_in >= 2) & (body_out >1/2 * body_in)) 

remove_index = df1[remove_index== True].index
remove_index.shape

(29345,)

In [235]:
df1[((head_in==1) & ( head_out == 3))]

,title,date,tags,section,text,url,head_in,head_out
485,MEMBERS GET RUNAROUND AT LUCILLE ROBERTS GYMS - CITY FINES CHAIN FOR BILLING WOES,"January 19, 1999",[],['news'],"TV ads for the Lucille Roberts gyms promote fun and fitness, but the Department of Consumer Affairs is slapping them with $150,000 in fines for holding members and their money hostage.\n\nCommissioner Jules Polonetsky is warning folks eager to work out at fitness facilities to watch out for bogus claims and know the facts before signing contracts.\n\n“Buying your membership today is very similar to buying a new car,” he said yesterday at the 92nd Street Y, as members worked out around him. “Consumers need to exercise caution and understand what the contract says before signing on the dotte...",https://nypost.com/1999/01/19/members-get-runaround-at-lucille-roberts-gyms-city-fines-chain-for-billing-woes/,{'department': 1},"{'tv': 1, 'fitness': 1, 'consumer': 1}"
4202,SPITZER: TIX BIZ ROTTEN WITH SCALPER HELPERS,"May 28, 1999",[],['news'],"ALBANY – A totally corrupt ticket industry is denying fairly priced tickets to sports, music and theater fans for nearly every popular event, state Attorney General Eliot Spitzer charged yesterday.\n\nIn a scathing report, Spitzer said his office found a “very large percentage” of seats are not available to the public for most events.\n\nThe attorney general found wrongdoing everywhere – promoters, performers, ticket brokers, box-office workers, Ticketmaster and nearly everyone else involved in selling tickets.\n\nDespite the massive fraud, Spitzer said his office will not be able to bring...",https://nypost.com/1999/05/28/spitzer-tix-biz-rotten-with-scalper-helpers/,{'corrupt': 1},"{'ticket': 2, 'sport': 1, 'music': 1}"
4440,THE 'VELVET FOG' RISES: MEL TORME DEAD AT 73,"June 6, 1999",[],['news'],"Mel Torme, the versatile jazz and pop singer and songwriter whose silky-smooth crooning earned him the nickname “The Velvet Fog,” died yesterday at age 73.\n\nTorme died from complications of a stroke he suffered in August 1996, his publicist Rob Wilcox said.\n\nThe singer’s fourth wife Ali and five children were by his side at UCLA Medical Center when he died at 1:45 a.m., Wilcox said.\n\n“He was a very special talent, one of the most talented individuals in our business,” said singer Vic Damone. “A composer-musician-arranger. He was so very bright.”\n\nTorme was mourned by friends includ...",https://nypost.com/1999/06/06/the-velvet-fog-rises-mel-torme-dead-at-73/,{'dead': 2},"{'jazz': 1, 'pop': 1, 'singer': 1}"
7845,"SORRY, GOVERNOR, BUT STATE DOES FUND B'KLYN MUSEUM; EXCLUSIVE","October 4, 1999",[],['news'],"HUNDREDS of thousands of state dollars are underwriting the Brooklyn Museum of Art’s explosive “Sensation” show, despite Gov. Pataki’s contentions to the contrary.\n\nOfficial state records reviewed by The Post reveal that more than $300,000 in state funds went to the museum last year, and will do so again this year.\n\nAnd since Pataki took office in 1995, the state has pumped nearly $2 million into the museum, according to records from the state comptroller and legislative fiscal committees.\n\nPataki has repeatedly ducked questions about whether he backs Mayor Giuliani’s effort to halt ...",https://nypost.com/1999/10/04/sorry-governor-but-state-does-fund-bklyn-museum-exclusive/,{'govern': 2},"{'fund': 2, 'dollar': 1, 'art': 1}"
8071,ONCE-DEAD METS HAVE GHOST OF A CHANCE ; ONCE-DEAD METS HAVE GHOST OF A CHANCE,"October 11, 1999",[],['news'],"DURING the regular season, the Mets lost nine of 12 games to the Atlanta Braves.\n\nThey were outscored 63-40. Their .279 team batting average plunged to .224 against Brave pitching. Their team 4.27 ERA soared to 5.23 against Brave batters.\n\nAnd when they absolutely, positively had to win in those dark days when it looked as if the last thing they would be doing was playing baseball in October, they lost. They lost five times in six games.\n\nIn that 10-day span, the Mets died.\n\nThe fa

In [222]:
df1.loc[remove_index]

,title,date,tags,section,text,url,head_in,head_out
0,TOP STATE AIDE IN MORALS BUST,"January 1, 1999",[],['news'],"ALBANY – A top aide to state Economic Development Director Charles Gargano has been busted by State Police on public-lewdness charges, officials said yesterday.\n\nDeputy Commissioner for Intergovernmental Relations Ronald O’Brien, 39, was picked up in a men’s room in the Empire State Plaza after he “displayed his genitals in a lewd manner” to an undercover state trooper, officials said.\n\nState Police Capt. Gerald Meyer said O’Brien was one of several people arrested Dec. 23 in an ongoing crackdown on homosexual activity in men’s rooms in and near the state Capitol. The crackdown has res...",https://nypost.com/1999/01/01/top-state-aide-in-morals-bust/,{},{'econom': 1}
4,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS ; 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square and rang in the last year of the millennium last night with the kind of raucous party that only New York can throw.\n\n“This is just the best place on earth to be!” said Greg Starrette, 19, a tourist from Tennessee who had no accommodation for New Year’s Eve except a patch of pavement in the Crossroads of the World.\n\n“We have nowhere to stay here, so we’re just going to hang around and watch the ball drop,” Starrette said, huddling against the cold.\n\nNot everyone was as excited as Sta...",https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp/,{},{'temperature': 1}
19,"COLD LANG SYNE FOR NEW YEAR'S REVELERS 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square last night with the kind of raucous party that only New York can throw.\n\nSang Lan, the Chinese gymnast paralyzed at last summer’s Goodwill Games, got the best view of the celebration – and the honor of triggering the New Year’s Eve ball drop.\n\n“I’m very excited,” Sang said, with a big smile. “I have never seen this on TV – this is my first time.”\n\nFrom a viewing platform at 46th Street, Sang flipped the switch that began the ball’s 60-second descent.\n\nAnd then 1999 began with a ba...",https://nypost.com/1999/01/01/cold-lang-syne-for-new-years-revelers-500000-cram-into-times-sq-despite-bone-chilling-temp-3/,{},{'temperature': 1}
20,'UNCLE MILTIE' LAUGHS IN FACE OF MILD STROKE,"January 1, 1999",[],['news'],"Legendary, 90-year-old comedian Milton Berle said yesterday he had a mild stroke last month, but now feels great and is looking forward to outlasting the millennium.\n\n“The doctors say I had a mild stroke, but I don’t feel it,” said Berle. “I have no fear of anything, and I’m gonna have a terrific new year.”\n\nBerle said the stroke occurred on Dec. 5 – and his doctors told him to “cool it, don’t work so hard.”\n\n“I feel good,” Berle said in Los Angeles. “But I don’t care what age you are – you get dizzy, you trip, grab hold of the furniture and you don’t reach it.\n\n“You tie your shoel...",https://nypost.com/1999/01/01/uncle-miltie-laughs-in-face-of-mild-stroke/,{},{'comed': 1}
23,"COLD LANG SYNE FOR; NEW YEAR'S REVELERS 500,000 CRAM INTO TIMES SQ., DESPITE BONE-CHILLING TEMP","January 1, 1999",[],['news'],"Happy New Year!\n\nClose to half a million revelers braved below-freezing temperatures in Times Square last night during the kind of raucous party that only New York can throw.\n\nSang Lan, the Chinese gymnast paralyzed at last summer’s Goodwill Games, got the best view of the celebration – and the honor of triggering the New Year’s Eve ball drop.\n\n“I’m very excited,” Sang said, with a big smile. “I have never seen this on TV – this is my first time.”\n\nWith Mayor Giuliani at her side on a viewing platform at 46th Street, Sang pushed the bi

In [223]:
df1 = df1[~df1.index.isin(remove_index)].reset_index(drop=True)
df1

,title,date,tags,section,text,url,head_in,head_out
0,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a...",https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,{},{}
1,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w...",https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'embassy ': 2, 'terror': 2, 'threat': 2}",{}
2,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai...",https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,{},{}
3,EXTRA TIME GIVES US ALL A 'SECOND' CHANCE,"January 1, 1999",[],['news'],"The stroke of midnight was a second late this year – thanks to a slowdown in the rotation of the Earth.\n\nThe International Earth Rotation Service inserted a leap-second before last night’s festivities to compensate for changes in the planet’s tidal patterns that have slowed its usual 365-day trip around the sun.\n\nThe leap second – the 20th to be added since 1972 – was added at 7 p.m. last night.\n\nThe decelerating Earth is nothing to be alarmed about, IERS officials said. The tidal effect does not mean the Earth is grinding to a halt, simply that we have become off-kilter from the unw...",https://nypost.com/1999/01/01/extra-time-gives-us-all-a-second-chance/,{},{}
4,CITY POLS VOW MIGHTY MOUSE RESPONSE IT'S WAR ON RESTAURANT RODENTS MIGHTY MOUSE CRACKDOWN,"January 1, 1999",[],['news'],"“I suspect this is a more widespread problem, and we’ve got to have a vigorous inspection system.”CITY COUNCILMANANDREW ERISTOFF\n\nCity officials yesterday vowed to wage a war on mice following The Post’s report of an infested Dunkin’ Donuts, saying more needs to be done to keep rodents out of restaurants.\n\n002 . 0000.00While calling for doubling the number of restaurant inspections each year, City Councilman Walter McCaffrey (D-Queens) also found some humor in the situation.\n\n“So that’s how they get the holes in the dough002 . 0000.00nuts,” he joked.\n\nMcCaffrey said the Dunkin’ Don...",https://nypost.com/1999/01/01/city-pols-vow-mighty-mouse-response-its-war-on-restaurant-rodents-mighty-mouse-crackdown/,"{'war': 2, 'suspect': 1}",{}
...,...,...,.

In [224]:
df1

,title,date,tags,section,text,url,head_in,head_out
0,IRAQ MOCKS 'EVIL ADULTERER' CLINTON,"January 1, 1999",[],['news'],"Iraq vowed yesterday to “liberate” its skies from U.S. and British jets that “support the most evil man in the world” – President Clinton.\n\nThe declaration, by Gen. Ali Hassan al-Majeed, a key figure in Saddam Hussein’s Revolutionary Command Council, followed two exchanges of missiles this week between U.S. warplanes and Iraqi anti-aircraft sites in “no-fly” zones.\n\nMajeed repeated Baghdad’s boast that “heroes of our air defenses” shot down a Western jet – while the United States ridiculed the claim as propaganda.\n\nIn a telegram to Saddam, Majeed said Iraq’s battered military would a...",https://nypost.com/1999/01/01/iraq-mocks-evil-adulterer-clinton/,{},{}
1,U.S. SHUTS TEL AVIV EMBASSY AFTER TERROR THREAT,"January 1, 1999",[],['news'],"JERUSALEM – The U.S. Embassy in Tel Aviv was abruptly shut yesterday after being warned of a “credible and specific” terrorist threat, authorities said.\n\nU.S. officials gave no details – and did not confirm an Israeli radio report that an anonymous caller threatened to blow up the embassy yesterday.\n\nThe caller was believed to be tied to Islamic fundamentalists, the radio report said.\n\nAmerican diplomatic missions throughout the world have been on alert since the Aug. 7 bombings of the U.S. embassies in Nairobi, Kenya and Dar es Salaam, Tanzania, which killed 224 people.\n\nConcern w...",https://nypost.com/1999/01/01/u-s-shuts-tel-aviv-embassy-after-terror-threat/,"{'embassy ': 2, 'terror': 2, 'threat': 2}",{}
2,"SOME HACKS JUST CAN'T BE HELPED, EXPERTS SAY","January 1, 1999",[],['news'],"New York audiences may be the most haphazard and impolite hackers in the world, but if they try to stifle their coughs, they’ll only make things worse, experts say.\n\n“Coughing seems to be a little like yawning – one person does it and then another thinks, ‘Oh, no – I’m next!'” said Manhattan psychotherapist Sheenah Hankin. “Coughing is a real panic reaction, and it’s something you can’t stop.”\n\nBut you can try to calm down, she suggested. “[Also], carry bottled water. Phlegm is water-soluble and if you take a swig, you’re going to clear your throat.”\n\nHankin, a native of England, sai...",https://nypost.com/1999/01/01/some-hacks-just-cant-be-helped-experts-say/,{},{}
3,EXTRA TIME GIVES US ALL A 'SECOND' CHANCE,"January 1, 1999",[],['news'],"The stroke of midnight was a second late this year – thanks to a slowdown in the rotation of the Earth.\n\nThe International Earth Rotation Service inserted a leap-second before last night’s festivities to compensate for changes in the planet’s tidal patterns that have slowed its usual 365-day trip around the sun.\n\nThe leap second – the 20th to be added since 1972 – was added at 7 p.m. last night.\n\nThe decelerating Earth is nothing to be alarmed about, IERS officials said. The tidal effect does not mean the Earth is grinding to a halt, simply that we have become off-kilter from the unw...",https://nypost.com/1999/01/01/extra-time-gives-us-all-a-second-chance/,{},{}
4,CITY POLS VOW MIGHTY MOUSE RESPONSE IT'S WAR ON RESTAURANT RODENTS MIGHTY MOUSE CRACKDOWN,"January 1, 1999",[],['news'],"“I suspect this is a more widespread problem, and we’ve got to have a vigorous inspection system.”CITY COUNCILMANANDREW ERISTOFF\n\nCity officials yesterday vowed to wage a war on mice following The Post’s report of an infested Dunkin’ Donuts, saying more needs to be done to keep rodents out of restaurants.\n\n002 . 0000.00While calling for doubling the number of restaurant inspections each year, City Councilman Walter McCaffrey (D-Queens) also found some humor in the situation.\n\n“So that’s how they get the holes in the dough002 . 0000.00nuts,” he joked.\n\nMcCaffrey said the Dunkin’ Don...",https://nypost.com/1999/01/01/city-pols-vow-mighty-mouse-response-its-war-on-restaurant-rodents-mighty-mouse-crackdown/,"{'war': 2, 'suspect': 1}",{}
...,...,...,.